In [ ]:
# %%
import pandas as pd
import numpy as np
import joblib
import warnings
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import os
import featuretools as ft
import shap
import lime
import lime.lime_tabular
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

# Suppress warnings
warnings.filterwarnings("ignore")

# ==== Paths ====
model_path = "D:\Python\Models\gpu_lightgbm_model.pkl"
scaler_path = "D:\Python\Models\scaler.pkl"
selected_features_path = "D:\Python\Models\selected_features.pkl"
poly_path = "D:\Python\Models\poly.pkl"
feature_defs_path = "D:\Python\Models\Feature_defs.pkl"
X_columns_path = "D:\Python\Models\X_columns.pkl"  # Contains (original_columns, ohe)
imputer_path = "D:\Python\Models\imputer.pkl"  # Added imputer path
test_data_path = "D:\Python\Models\Cleaned_Aligned_Test_Dataset.csv"
predictions_output_path = "D:\Python\Models\predictions.csv"

# ==== Load Model and Preprocessing Objects ====
print("🔄 Loading model and preprocessing objects...")
model = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_features = joblib.load(selected_features_path)
poly = joblib.load(poly_path)

# FIX: Load feature definitions with correct encoding
print("🔄 Loading feature definitions...")
with open(feature_defs_path, "r", encoding="utf-8") as f:
    features_content = f.read()
feature_defs = ft.load_features(features_content)

original_columns, ohe = joblib.load(X_columns_path)  # Split tuple [[1]][[2]]
imputer = joblib.load(imputer_path)  # Load imputer [[3]]

# ==== Load Test Data ====
print("📊 Loading test data...")
test_data = pd.read_csv(test_data_path)
target_column = 'target'
if target_column not in test_data.columns:
    raise KeyError("❌ 'target' column is missing in test data!")
X_test = test_data.drop(columns=[target_column])
y_test = test_data[target_column]

# ==== Handle Categorical Variables ====
print("🧠 Encoding categorical variables using training OHE...")
cat_cols = X_test.select_dtypes(include=['object', 'category']).columns.tolist()
if cat_cols:
    X_test_cat = ohe.transform(X_test[cat_cols])
    X_test = pd.concat([
        X_test.drop(columns=cat_cols),
        pd.DataFrame(X_test_cat, columns=ohe.get_feature_names_out(cat_cols), index=X_test.index)
    ], axis=1)
# Align with training columns
X_test = X_test.reindex(columns=original_columns, fill_value=0)  # Use correct columns [[4]]

# ==== Generate Polynomial Features ====
print("🛠️ Generating polynomial features...")
X_poly_test = pd.DataFrame(poly.transform(X_test), columns=poly.get_feature_names_out(X_test.columns))

# ==== Feature Engineering with Featuretools ====
print("🧠 Applying Featuretools transformations...")
X_test_feat = X_test.copy()
X_test_feat['idx'] = X_test_feat.index
es_test = ft.EntitySet(id="test_data")
es_test.add_dataframe(dataframe_name="data", dataframe=X_test_feat, index="idx")
X_engineered_test = ft.calculate_feature_matrix(feature_defs, entityset=es_test)
new_features = [col for col in X_engineered_test.columns if col not in X_test.columns]
X_engineered_test = X_engineered_test[new_features]

# ==== Combine Features ====
print("🔗 Combining features...")
X_final_test = pd.concat([X_poly_test, X_engineered_test], axis=1)

# ==== Impute Missing Values ====
print("🧩 Imputing missing values using training imputer...")
X_final_test = pd.DataFrame(imputer.transform(X_final_test), columns=X_final_test.columns)  # Apply imputation [[3]]

# ==== Scale Test Data ====
print("⚙️ Scaling features...")
X_final_test_scaled = scaler.transform(X_final_test)

# ==== Select Features ====
print("🔍 Selecting features...")
# Use column indices from original (pre-scaled) data
feature_indices = [X_final_test.columns.get_loc(col) for col in selected_features]
X_selected_test = X_final_test_scaled[:, feature_indices]

# ==== Make Predictions ====
print("🔮 Making predictions on test data...")
y_pred = model.predict(X_selected_test)

# ==== Evaluate Predictions ====
print("📊 Evaluating predictions...")
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)
print("R2 Score:", r2)

# ==== Save Predictions ====
print("💾 Saving predictions...")
test_data['predictions'] = y_pred
test_data.to_csv(predictions_output_path, index=False)
print("✅ Predictions saved to:", predictions_output_path)